## **2. Business Understanding**

### **2.2 Data Scope**

- **Origin**: Collected from the UCI Machine Learning Repository ([ID: 352](https://archive.ics.uci.edu/dataset/352/online+retail)). It's a real-world transactional dataset donated for research.
- **Format**: Single CSV file (Online_Retail.csv).  
- **Scope**: Contains all transactions for a 1-year period (Dec 2010 - Dec 2011) for a UK-based online retailer selling unique gifts, with many wholesale customers.
- **Business Relevance**: This dataset directly supports Shoppy's needs by providing:
    - Complete transaction history for customer behavior analysis
    - Variables essential for segmentation (purchase frequency, monetary value, recency)  
    - Customer identifiers for targeted campaign implementation
    - Temporal data for understanding engagement patterns

### **2.3 Data Loading and Initial Exploration**

In [1]:
# Import libraries and load data
import pandas as pd
import sys
import os

# Add src directory to path for importing our module
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'src'))

# Load raw data
df = pd.read_csv('../data/raw/Online_Retail.csv')
print(f"Dataset shape: {df.shape}")
print(f"Dataset size: {df.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")
df.info()



Dataset shape: (541909, 8)
Dataset size: 173.65 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


### **2.4 Data Assessment**

In [3]:
print("\n" + "="*50)
print("DATA COLUMN DESCRIPTIONS:")
print("="*50)

data_description = {
    'InvoiceNo': 'Transaction identifier (6-digit integral number)',
    'StockCode': 'Product identifier (5-digit integral number)',
    'Description': 'Product description (text)',
    'Quantity': 'Items purchased per transaction (numeric)',
    'InvoiceDate': 'Transaction timestamp (datetime)',
    'UnitPrice': 'Price per unit in sterling (numeric)',
    'CustomerID': 'Customer identifier (5-digit integral number)',
    'Country': 'Customer location (country name)'
}

for col, desc in data_description.items():
    print(f"• {col}: {desc}")


DATA COLUMN DESCRIPTIONS:
• InvoiceNo: Transaction identifier (6-digit integral number)
• StockCode: Product identifier (5-digit integral number)
• Description: Product description (text)
• Quantity: Items purchased per transaction (numeric)
• InvoiceDate: Transaction timestamp (datetime)
• UnitPrice: Price per unit in sterling (numeric)
• CustomerID: Customer identifier (5-digit integral number)
• Country: Customer location (country name)


### **2.5 Data Quality Analysis**

In [4]:
# Check for missing values using our modular function
from data_preprocessing import check_missing_values

missing_summary = check_missing_values(df)
print("MISSING VALUES ANALYSIS:")
print(missing_summary)

print(f"\nMISSING DATA IMPACT:")
print(f"• Description missing: {missing_summary.loc['Description', 'Missing_Percent']:.2f}% of transactions")
print(f"• CustomerID missing: {missing_summary.loc['CustomerID', 'Missing_Percent']:.2f}% of transactions")
print(f"• Transactions without CustomerID cannot be used for customer-level analysis")

# Note: Function available in src/data_preprocessing.py

MISSING VALUES ANALYSIS:
             Missing_Count  Missing_Percent
Description           1454             0.27
CustomerID          135080            24.93

MISSING DATA IMPACT:
• Description missing: 0.27% of transactions
• CustomerID missing: 24.93% of transactions
• Transactions without CustomerID cannot be used for customer-level analysis


### **2.6 Statistical Summary**

In [5]:
# Display statistical summary for numerical columns
print("STATISTICAL SUMMARY:")
print(df[['Quantity', 'UnitPrice']].describe())

print("\n" + "="*50)
print("KEY INSIGHTS FROM STATISTICS:")
print("="*50)

# Calculate additional insights
total_transactions = len(df)
unique_customers = df['CustomerID'].nunique()
unique_products = df['StockCode'].nunique()
date_range = f"{df['InvoiceDate'].min()} to {df['InvoiceDate'].max()}"

print(f"• Total Transactions: {total_transactions:,}")
print(f"• Unique Customers: {unique_customers:,}")
print(f"• Unique Products: {unique_products:,}")
print(f"• Date Range: {date_range}")
print(f"• Average Quantity per Transaction: {df['Quantity'].mean():.2f}")
print(f"• Average Unit Price: £{df['UnitPrice'].mean():.2f}")

STATISTICAL SUMMARY:
            Quantity      UnitPrice
count  541909.000000  541909.000000
mean        9.552250       4.611114
std       218.081158      96.759853
min    -80995.000000  -11062.060000
25%         1.000000       1.250000
50%         3.000000       2.080000
75%        10.000000       4.130000
max     80995.000000   38970.000000

KEY INSIGHTS FROM STATISTICS:
• Total Transactions: 541,909
• Unique Customers: 4,372
• Unique Products: 4,070
• Date Range: 01/02/2011 08:23 to 31/10/2011 17:19
• Average Quantity per Transaction: 9.55
• Average Unit Price: £4.61


### **2.8 Geographic Distribution**

In [6]:
# Analyze country distribution
country_counts = df['Country'].value_counts()
total_countries = len(country_counts)

print(f"GEOGRAPHIC DISTRIBUTION:")
print(f"• Total Countries: {total_countries}")
print(f"• Primary Market: {country_counts.index[0]} ({country_counts.iloc[0]:,} transactions)")
print(f"• Secondary Markets: {country_counts.index[1]} ({country_counts.iloc[1]:,} transactions)")

print(f"\nTOP 10 COUNTRIES BY TRANSACTION VOLUME:")
print(country_counts.head(10))

# Calculate UK dominance
uk_percentage = (country_counts['United Kingdom'] / len(df)) * 100
print(f"\nUK Market Dominance: {uk_percentage:.1f}% of all transactions")

GEOGRAPHIC DISTRIBUTION:
• Total Countries: 38
• Primary Market: United Kingdom (495,478 transactions)
• Secondary Markets: Germany (9,495 transactions)

TOP 10 COUNTRIES BY TRANSACTION VOLUME:
Country
United Kingdom    495478
Germany             9495
France              8557
EIRE                8196
Spain               2533
Netherlands         2371
Belgium             2069
Switzerland         2002
Portugal            1519
Australia           1259
Name: count, dtype: int64

UK Market Dominance: 91.4% of all transactions


### **2.9 Business Impact Assessment**

In [7]:

print("BUSINESS IMPACT ASSESSMENT:")
print("="*50)

# Assess data quality impact on business objectives
print("DATA QUALITY IMPACT ON SEGMENTATION:")

# Customer coverage
customers_with_id = df['CustomerID'].notna().sum()
customer_coverage = (customers_with_id / len(df)) * 100
print(f"• Customer Identification Coverage: {customer_coverage:.1f}%")
print(f"• Transactions Available for Segmentation: {customers_with_id:,}")

# Transaction completeness
complete_transactions = df.dropna(subset=['Description', 'CustomerID']).shape[0]
transaction_completeness = (complete_transactions / len(df)) * 100
print(f"• Complete Transaction Records: {transaction_completeness:.1f}%")

# Business value indicators
print(f"\nBUSINESS VALUE INDICATORS:")
print(f"• Average Revenue per Transaction: £{df['UnitPrice'].mean() * df['Quantity'].mean():.2f}")
print(f"• High-Value Transactions (>£100): {(df['UnitPrice'] * df['Quantity'] > 100).sum():,}")
print(f"• International Business: {(1 - uk_percentage/100):.1%} of transactions")

BUSINESS IMPACT ASSESSMENT:
DATA QUALITY IMPACT ON SEGMENTATION:
• Customer Identification Coverage: 75.1%
• Transactions Available for Segmentation: 406,829
• Complete Transaction Records: 75.1%

BUSINESS VALUE INDICATORS:
• Average Revenue per Transaction: £44.05
• High-Value Transactions (>£100): 13,554
• International Business: 8.6% of transactions


### **2.10 Data Preparation Requirements**

In [8]:

print("DATA PREPARATION REQUIREMENTS:")
print("="*50)

print("REQUIRED TRANSFORMATIONS:")
print("1. Missing Value Handling:")
print("   • Remove rows with missing Description (affects product analysis)")
print("   • Handle missing CustomerID (fill with 'Guest' for transaction-level analysis)")

print("\n2. Data Type Corrections:")
print("   • Convert InvoiceDate to datetime format")
print("   • Convert CustomerID to string/object type")
print("   • Ensure proper numeric types for Quantity and UnitPrice")

print("\n3. Data Cleaning:")
print("   • Remove duplicate transactions")
print("   • Handle cancelled orders (negative quantities)")
print("   • Remove outliers in UnitPrice and Quantity")

print("\n4. Feature Engineering:")
print("   • Create Revenue column (Quantity × UnitPrice)")
print("   • Extract temporal features (day, month, season)")
print("   • Prepare for RFM analysis")

print(f"\nESTIMATED DATA RETENTION:")
print(f"• After removing missing CustomerID: {customer_coverage:.1f}%")
print(f"• After removing duplicates: ~95-98%")
print(f"• After outlier removal: ~85-90%")
print(f"• Final usable data: ~80-85% of original")

DATA PREPARATION REQUIREMENTS:
REQUIRED TRANSFORMATIONS:
1. Missing Value Handling:
   • Remove rows with missing Description (affects product analysis)
   • Handle missing CustomerID (fill with 'Guest' for transaction-level analysis)

2. Data Type Corrections:
   • Convert InvoiceDate to datetime format
   • Convert CustomerID to string/object type
   • Ensure proper numeric types for Quantity and UnitPrice

3. Data Cleaning:
   • Remove duplicate transactions
   • Handle cancelled orders (negative quantities)
   • Remove outliers in UnitPrice and Quantity

4. Feature Engineering:
   • Create Revenue column (Quantity × UnitPrice)
   • Extract temporal features (day, month, season)
   • Prepare for RFM analysis

ESTIMATED DATA RETENTION:
• After removing missing CustomerID: 75.1%
• After removing duplicates: ~95-98%
• After outlier removal: ~85-90%
• Final usable data: ~80-85% of original


### **2.11 Configuration Reference**

In [10]:
# Data source information is stored in src/config.py
from config import DATA_SOURCE

print("DATA SOURCE CONFIGURATION:")
print("="*50)
for key, value in DATA_SOURCE.items():
    print(f"• {key.replace('_', ' ').title()}: {value}")

print(f"\nACCESSING CONFIGURATION IN CODE:")
print("from src.config import DATA_SOURCE")
print(DATA_SOURCE['origin'])
print(DATA_SOURCE['time_period'])

DATA SOURCE CONFIGURATION:
• Origin: UCI Machine Learning Repository
• Dataset Id: 352
• Url: https://archive.ics.uci.edu/dataset/352/online+retail
• Time Period: Dec 2010 - Dec 2011
• Business Type: UK-based online retailer
• Description: Complete transaction history for customer behavior analysis

ACCESSING CONFIGURATION IN CODE:
from src.config import DATA_SOURCE
UCI Machine Learning Repository
Dec 2010 - Dec 2011


### **2.12 Summary and Next Steps**

In [11]:
print("DATA UNDERSTANDING SUMMARY:")
print("="*50)

print("DATASET CHARACTERISTICS:")
print(f"• Dataset contains {len(df):,} transactions from {total_countries} countries")
print(f"• Time period: Dec 2010 - Dec 2011 (12 months)")
print(f"• {unique_customers:,} unique customers with {unique_products:,} unique products")
print(f"• UK-dominated market ({uk_percentage:.1f}% of transactions)")

print(f"\nDATA QUALITY ASSESSMENT:")
print(f"• Missing CustomerID: {missing_summary.loc['CustomerID', 'Missing_Percent']:.1f}% (major concern)")
print(f"• Missing Description: {missing_summary.loc['Description', 'Missing_Percent']:.1f}% (minor concern)")
print(f"• Data type issues require correction before analysis")

print(f"\nBUSINESS READINESS:")
print("✅ Sufficient transaction volume for segmentation")
print("✅ Complete temporal data for RFM analysis")
print("✅ Product diversity for behavioral analysis")
print("⚠️  Customer ID coverage needs attention")
print("⚠️  Data cleaning required before modeling")

print(f"\nNEXT STEPS:")
print("1. Execute data preprocessing pipeline (Data_Preprocessing.ipynb)")
print("2. Create customer-level features (Feature_Engineering.ipynb)")
print("3. Perform exploratory analysis (EDA.ipynb)")
print("4. Develop segmentation model (Modeling.ipynb)")

print(f"\nNote: Use src/data_preprocessing.py for production data cleaning.")

DATA UNDERSTANDING SUMMARY:
DATASET CHARACTERISTICS:
• Dataset contains 541,909 transactions from 38 countries
• Time period: Dec 2010 - Dec 2011 (12 months)
• 4,372 unique customers with 4,070 unique products
• UK-dominated market (91.4% of transactions)

DATA QUALITY ASSESSMENT:
• Missing CustomerID: 24.9% (major concern)
• Missing Description: 0.3% (minor concern)
• Data type issues require correction before analysis

BUSINESS READINESS:
✅ Sufficient transaction volume for segmentation
✅ Complete temporal data for RFM analysis
✅ Product diversity for behavioral analysis
⚠️  Customer ID coverage needs attention
⚠️  Data cleaning required before modeling

NEXT STEPS:
1. Execute data preprocessing pipeline (Data_Preprocessing.ipynb)
2. Create customer-level features (Feature_Engineering.ipynb)
3. Perform exploratory analysis (EDA.ipynb)
4. Develop segmentation model (Modeling.ipynb)

Note: Use src/data_preprocessing.py for production data cleaning.


### **2.7 Data Type Analysis**

In [12]:

print("CURRENT DATA TYPES:")
print(df.dtypes)

print("\n" + "="*50)
print("DATA TYPE ISSUES:")
print("="*50)

# Identify data type issues
data_issues = []

if df['InvoiceDate'].dtype == 'object':
    data_issues.append("• InvoiceDate stored as object instead of datetime")

if df['CustomerID'].dtype == 'float64':
    data_issues.append("• CustomerID stored as float instead of string/object")

# Check for negative values (cancelled orders)
negative_quantity = (df['Quantity'] < 0).sum()
negative_price = (df['UnitPrice'] < 0).sum()

if negative_quantity > 0:
    data_issues.append(f"• {negative_quantity} transactions with negative quantities (cancelled orders)")

if negative_price > 0:
    data_issues.append(f"• {negative_price} transactions with negative prices")

if data_issues:
    for issue in data_issues:
        print(issue)
else:
    print("No major data type issues detected.")

CURRENT DATA TYPES:
InvoiceNo       object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID     float64
Country         object
dtype: object

DATA TYPE ISSUES:
• InvoiceDate stored as object instead of datetime
• CustomerID stored as float instead of string/object
• 10624 transactions with negative quantities (cancelled orders)
• 2 transactions with negative prices
